In [1]:
%load_ext autoreload
%autoreload 2
#%load_ext blackcellmagic

In [ ]:
# Install graphein from master for bleeding-edge additions
!pip install git+https://github.com/a-r-j/graphein

In [2]:
# Notebook dev tools
from typing import *
import copy

# General tools
import numpy as np

# Pytorch tools
import torch
import torch.nn as nn
from hydra.compose import GlobalHydra
from proteinworkshop.utils.notebook import *

init_hydra_singleton(reload=True)

path = HYDRA_CONFIG_PATH
rel_path = os.path.relpath(path, start=".")

GlobalHydra.instance().clear()
hydra.initialize(rel_path)

cfg = hydra.compose("template", overrides=["dataset=afdb_swissprot_v4"], return_hydra_config=False)

/home/atj39/github/protein_workshop/src/utils/notebook.py:38: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(rel_path)
2023-04-01 00:12:17.326 | INFO     | src.utils.notebook:init_hydra_singleton:39 - Hydra initialised at /home/atj39/github/protein_workshop/configs.
/tmp/ipykernel_2160491/3798039777.py:20: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(rel_path)


### Load a dataset

Can switch out for another by replacing the dataset arg in overrides:

`cfg = hydra.compose("template", overrides=["dataset=afdb_swissprot_v4"], return_hydra_config=False)`

In [5]:
from proteinworkshop.datasets.factory import get_dataset

datamodule = get_dataset(cfg)
datamodule.setup("train")
dl = datamodule.train_dataloader()

100%|██████████| 542378/542378 [00:00<00:00, 3186587.18it/s]
Processing...
Done!


In [17]:
for i in dl:
    print(i)
    break

/home/atj39/mambaforge/envs/workshop/lib/python3.9/site-packages/torch_geometric/data/collate.py:145: UserWarning:

TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()

/home/atj39/mambaforge/envs/workshop/lib/python3.9/site-packages/torch_geometric/data/collate.py:145: UserWarning:

TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()

/home/atj39/mambaforge/envs/workshop/lib/python3.9/site-packages/torch_geometric/data/collate.py:145: UserWarning:

TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage c

DataBatch(chains=[10380], fill_value=[32], coords=[10380, 37, 3], residue_type=[10380], residue_id=[32], residues=[32], id=[32], atom_list=[32])


/home/atj39/mambaforge/envs/workshop/lib/python3.9/site-packages/torch_geometric/data/collate.py:145: UserWarning:

TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()

/home/atj39/mambaforge/envs/workshop/lib/python3.9/site-packages/torch_geometric/data/collate.py:145: UserWarning:

TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()

/home/atj39/mambaforge/envs/workshop/lib/python3.9/site-packages/torch_geometric/data/collate.py:145: UserWarning:

TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage c

In [18]:
i

DataBatch(chains=[10380], fill_value=[32], coords=[10380, 37, 3], residue_type=[10380], residue_id=[32], residues=[32], id=[32], atom_list=[32])

### Featurise a batch

In [19]:
from proteinworkshop.features.factory import get_features
print(cfg.features)


data = get_features(i, cfg)
data

{'representation': 'CA', 'node_features': ['amino_acid_one_hot'], 'edge_types': ['knn_10', 'eps_8'], 'edge_features': ['edge_distance', 'edge_angle', 'edge_torsion', 'edge_type_one_hot']}
[tensor[10380, 23] n=238740 x∈[0., 1.000] μ=0.043 σ=0.204]


DataBatch(chains=[10380], fill_value=[32], coords=[10380, 3], residue_type=[10380], residue_id=[32], residues=[32], id=[32], atom_list=[32], pos=[10380, 37, 3], node_features=[10380, 23], edge_index=[2, 396253], edge_type=[1, 396253])

### Define a new encoder

In [ ]:
import torch.nn
from graphein.protein.tensor.data import ProteinBatch
from torch_geometric.data import Batch
from proteinworkshop.types import EncoderOutput


class MyNewProteinEncoder(nn.Module):
    def __init__(self):
        pass

    def foward(self, batch: Union[Batch, ProteinBatch]) -> EncoderOutput:
        output = {}
        # Do stuff...
        output["node_embeddings"] = torch.randn(10, 10)
        output["graph_embeddings"] = self.aggregation(output["node_embeddings"])
        return output